# 1\. Descrição e metódo de cálculo
A ideia é analisar os dados de todos os anos da Formula 1 e definir o Campeão de todos os tempos. 
Será utilizada uma base de dados de 1950 até 2020, porém o sistema de pontuação será utilizado do ano de 2020, conforme lista abaixo.
1. 25 pontos;
2. 18 pontos;
3. 15 pontos;
4. 12 pontos;
5. 10 pontos;
6. 8 pontos;
7. 6 pontos;
8. 4 pontos;
9. 2 pontos;
10. 1 ponto.

- Não será utilizado o ponto extra de volta mais rápida, pois o dataset esta incompleto.

# 2\. Montagem da Estrutura

Importação de Bibliotecas e Dataset

## 2.1. Importação de Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 2.2. Carga de Dados

In [ ]:
df_drivers = pd.read_csv('../input/formula-1-world-championship-1950-2020/drivers.csv', na_values='\\N')
df_teams = pd.read_csv('../input/formula-1-world-championship-1950-2020/constructors.csv', na_values='\\N')
df_results = pd.read_csv('../input/formula-1-world-championship-1950-2020/results.csv', na_values='\\N')
df_races = pd.read_csv('../input/formula-1-world-championship-1950-2020/races.csv', na_values='\\N')

# 3\. Exploração dos Dados

Visualização e tratamento dos dados do Dataset, além de separar os dados que serão utilizados.

## 3.1. Visualização dos dados

In [ ]:
# Pilotos/Competidores
df_drivers.head(n=20)

In [ ]:
# Times/Equipes
df_teams.head(n=20)

In [ ]:
# Corridas
df_races.head(n=20)

In [ ]:
# Resultados
df_results.head(n=20)

## 3.2. Tratamento dos Dados

Montagem da tabela principal de trabalho.

In [ ]:
# Geração dos data frames apenas com as colunas e dados que interessa.
col_list_drivers = ['driverId','forename','surname','code']
df_drivers_cl = df_drivers.filter(col_list_drivers, axis=1)

col_list_teams = ['constructorId','name']
df_teams_cl = df_teams.filter(col_list_teams, axis=1)

col_list_races = ['raceId','year','round']
df_races_cl = df_races.filter(col_list_races, axis=1) 

#Além de selecionar as colunas que fazem sentido para a analise, selecionei apenas as linhas em que o piloto ficou entre as 10 primeiras colocações.
col_list_results = ['raceId','driverId','constructorId','position',]
df_results_cl = df_results[df_results['position'] < 11].filter(col_list_results, axis=1)

In [ ]:
#Verificação dos data frames criados anteriormente
print('Drivers')
print(df_drivers_cl.head(n=5))
print('')
print('Teams')
print(df_teams_cl.head(n=5))
print('')
print('Races')
print(df_races_cl.head(n=5))
print('')
print('Results')
print(df_results_cl.head(n=20))

# 4\. Análise dos dados

Analise dos dados e levantamento dos primeiros dados conforme metodo de calculo.

## 4.1. Quantidade de Etapas por Temporada

O calculo será feito por etapas, em um primeiro momento é importante separar a quantidade de etapas de cada temporada.

In [ ]:
#Visualização da quantidade de Etapas por ano
sns.set_theme(style='whitegrid')
fig, axis = plt.subplots(1,1, figsize=(20, 5), sharex=True)

df_races_plot = df_races_cl['year'].value_counts().reset_index()
df_races_plot.columns = ['Year', 'Races']
df_races_plot.rename(columns={'Year' : 'Races'})
df_races_plot.sort_values(by=['Year'], inplace=True)
df_races_plot.sort_values(by=['Year'])

graph = sns.barplot(x=df_races_plot['Year'], y=df_races_plot['Races'], palette="crest")
graph.set(title='Qty races per Year', xlabel='Years', ylabel='Races')
graph.set_xticklabels(labels=graph.get_xticklabels(), rotation=90)

max = df_races_plot.describe() * 1.1
max_y = 0

_, max_y_f = graph.get_ylim()
max_y = max_y_f if max_y_f > max_y else max_y
graph.set(ylim=(0, max_y))

fig.show()
df_races_plot.transpose()

## 4.2. Total de pontos por temporada

Visualização do total de pontos de cada piloto e equipe por temporada.

In [ ]:
# Mudança dos valores de position e rank de Float para Int
float_to_int = lambda value: int(value)

test_values = [10.0, 55.0, 2.0, 6.0]
values_int = list(map(float_to_int, test_values))

print(test_values)
print(values_int)

In [ ]:
df_results_cl.dtypes

In [ ]:
df_results_cl['position'] = df_results_cl['position'].apply(float_to_int)
#df_results_cl['rank'] = df_results_cl['rank'].apply(float_to_int)

In [ ]:
df_results_cl.dtypes

In [ ]:
# Adição da coluna com quantidade de pontos

points = [1, 25, 18, 15, 12, 10, 8, 6, 4, 2, 1]
df_results_cl['points'] = 0
df_results_cl['year'] = 0

for index, result in df_results_cl.iterrows():
    result['points'] = points[result['position']]

for index, result in df_results_cl.iterrows():
    
    raceId = int(result['raceId'])
    year = int(df_races_cl[df_races_cl['raceId'] == raceId]['year'])
    result['year'] = year
        
df_results_cl
    

In [ ]:
# Criação da listagem de anos
def ColumnToList(dataframe, array):
    for n in dataframe.sort_values():
        if n not in array:
            array.append(n)

            
years = []
ColumnToList(df_races_cl['year'], years)            
print(years)

In [ ]:
# Criação do dataset de pilotos x anos x pontos que será utilizado nos próximos graficos
df_drivers_years_points = df_results_cl.filter(['driverId','constructorId','points','year'], axis=1)
points_drivers_array = []
points_teams_array=[]

for year in years:
    
    points_year = df_drivers_years_points[df_drivers_years_points['year'] == year]
    for index, driver in df_drivers_cl.iterrows():
        
        array = []
        driver_name = f'''{driver['forename']} {driver['surname']}'''
        array.extend([driver_name, driver['code']])
        array.append(year)
        array.append(points_year[points_year['driverId'] == driver['driverId']].sum()['points'])
        points_drivers_array.append(array)
    
    for index, team in df_teams_cl.iterrows():
        
        array = []
        array.extend([team['name'], year])
        array.append(points_year[points_year['constructorId'] == team['constructorId']].sum()['points'])
        points_teams_array.append(array)

df_drivers_points = pd.DataFrame(points_drivers_array, columns=('name','code','year','points'))
df_teams_points = pd.DataFrame(points_teams_array, columns=('name','year','points'))


In [ ]:
# Visualização do dataframe dos piltos. Altere o valor de 1992 com o ano desejado.

view_drivers = df_drivers_points[df_drivers_points['year'] == 1992]
view_drivers = view_drivers[view_drivers['points'] > 0]
view_drivers[view_drivers['year'] == 1992].sort_values(by=['points'], ascending=False)


In [ ]:
# Visualização do dataframe das equipes. Altere o valor de 1992 com o ano desejado.

view_teams = df_teams_points[df_teams_points['year'] == 1992]
view_teams = view_teams[view_teams['points'] > 0]
view_teams[view_teams['year'] == 1992].sort_values(by=['points'], ascending=False)


In [ ]:
# Criação dos gráficos
sns.set_theme(style='whitegrid')

axis = 0
max_y = 0
columns = 2
rows = int(len(years) / columns)
view, axes = plt.subplots(rows,columns, figsize=(30, 300), sharex=False)
plt.subplots_adjust(hspace = 0.8)
axes = axes.flatten()

for year in years:
    
    plot = df_drivers_points[df_drivers_points['year'] == year]
    plot = plot[plot['points'] > 0].sort_values(by=['points'], ascending=False, axis=0)
    
    fig = sns.barplot(x=plot['name'], y=plot['points'], ax=axes[axis], ci=None, palette="crest")
    fig.set(title=f'Standings of F1 {year}', xlabel='Drivers', ylabel='Points')
    fig.set_xticklabels(labels=fig.get_xticklabels(), rotation=90)
    
    axis += 1
    max = plot.describe() * 1.1
    
    _, max_y_f = fig.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    fig.set(ylim=(0, max_y))

    
view.show()

In [ ]:
# Criação dos gráficos
sns.set_theme(style='whitegrid')

axis = 0
max_y = 0
columns = 2
rows = int(len(years) / columns)
view, axes = plt.subplots(rows,columns, figsize=(30, 300), sharex=False)
plt.subplots_adjust(hspace = 0.4)
axes = axes.flatten()

for year in years:
    
    plot = df_teams_points[df_teams_points['year'] == year]
    plot = plot[plot['points'] > 0].sort_values(by=['points'], ascending=False, axis=0)
    
    fig = sns.barplot(x=plot['name'], y=plot['points'], ax=axes[axis], ci=None, palette="crest")
    fig.set(title=f'Standings of F1 {year}', xlabel='Teams', ylabel='Points')
    fig.set_xticklabels(labels=fig.get_xticklabels(), rotation=90)
    
    axis += 1
    max = plot.describe() * 1.1
    
    _, max_y_f = fig.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    fig.set(ylim=(0, max_y))

    
view.show()

# 5\. Pontos totais

Calculos e gráficos da soma de todos os pontos de todos os anos.

In [ ]:
#Preparação dos dataframes

teams_list = []
drivers_list = []

ColumnToList(df_teams_points['name'], teams_list)
ColumnToList(df_drivers_points['name'], drivers_list)

def TotalPoints(dataframe, column_name, list):
    array_df = []
    for n in list:
        
        array = []
        array.extend([n, dataframe[dataframe[column_name] == n].sum()['points']])
        array_df.append(array)
        
    return pd.DataFrame(array_df, columns=('name', 'points'))

df_total_teams_points = TotalPoints(df_teams_points, 'name', teams_list)
df_total_drivers_points = TotalPoints(df_drivers_points, 'name', drivers_list)


In [ ]:
# Criação dos graficos

dataframes = [df_total_drivers_points, df_total_teams_points]
sns.set_theme(style='whitegrid')

axis = 0
max_y = 0
view, axes = plt.subplots(2,1, figsize=(30, 40), sharex=False)
plt.subplots_adjust(hspace = 0.2)
axes = axes.flatten()

for df in dataframes:
    
    plot = df[df['points'] > 200].sort_values(by=['points'], ascending=False, axis=0)
    
    fig = sns.barplot(x=plot['name'], y=plot['points'], ax=axes[axis], ci=None, palette="crest")
    fig.set(title=f'Total points of F1', xlabel='Names', ylabel='Points')
    fig.set_xticklabels(labels=fig.get_xticklabels(), rotation=90)
    
    axis += 1
    max = plot.describe() * 1.1
    
    _, max_y_f = fig.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    fig.set(ylim=(0, max_y))

    
view.show()